In [2]:
import dask_awkward as dak
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from distributed import LocalCluster, Client
import numpy as np


In [3]:
fname = "root://eos.cms.rcac.purdue.edu//store/mc/RunIISummer20UL18NanoAODv9/GluGluHToMuMu_M125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2810000/C4DAB63C-E2A1-A541-93A8-3F46315E362C.root"

file_dict = {"root://eos.cms.rcac.purdue.edu:1094///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-100to200_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/40000/AA6F89B0-EDAA-3942-A3BB-BC3709722EB4.root": {"object_path": "Events", "steps": [[0, 10], [10, 20]], "uuid": "b16b5dea-fbcd-11ed-bae7-a2a0b8bcbeef"}}
# file_dict = {"root://eos.cms.rcac.purdue.edu//store/mc/RunIIAutumn18NanoAODv6/DYJetsToLL_M-105To160_TuneCP5_PSweights_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/Nano25Oct2019_102X_upgrade2018_realistic_v20-v1/100000/1E5C1F4B-FE04-624F-82FD-C4273792B1D1.root": {"object_path": "Events", "steps": [[0, 1000], [1000, 2000]]}}


client = Client(n_workers=1,  threads_per_worker=1, processes=True)
events = NanoEventsFactory.from_root(
    file_dict,
    schemaclass=NanoAODSchema,
).events()

/depot/cms/kernels/root632/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43139 instead
  warnings.warn(
/depot/cms/kernels/root632/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/root632/lib/python3.12/site-packages/c

In [4]:
events = events.compute()

/depot/cms/kernels/root632/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [9]:
numevents = len(events)
mu1 = events.Muon[:,:1]
mu1["pt_raw"] = mu1.pt
pass_leading_pt = np.zeros(numevents, dtype=bool)
has_muons = ak.num(mu1.pt, axis=1)


<Array [0, 0, 1, 1, 0, 1, 0, 0, ..., 0, 1, 1, 0, 1, 0, 0, 1] type='20 * int64'>

In [10]:
pass_leading_pt[has_muons>0] = (mu1.pt_raw>26)[has_muons>0] 
print(pass_leading_pt)
# # All L3 trigger muons
# df.TrigObj['mass'] = df.TrigObj.zeros_like()
# df.TrigObj = df.TrigObj[(df.TrigObj.id == 13) | (df.TrigObj.id == -13)]

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [134]:
ak.all(ak.num(jets.genJetIdx, axis=1) == ak.num(jets.genJetIdxG, axis=1))

True

In [6]:
def delta_r(eta1, eta2, phi1, phi2):
    deta = abs(eta1 - eta2)
    dphi = abs(np.mod(phi1 - phi2 + np.pi, 2 * np.pi) - np.pi)
    dr = np.sqrt(deta**2 + dphi**2)
    return deta, dphi, dr


def checkGenIdxRecommended(events):
    delta_rs = events.Jet.delta_r(events.Jet.matched_gen) 
    delta_rs =  delta_rs.compute()
    # print(delta_r_test)
    print(f"checkGenIdxRecommended: {ak.all(delta_rs <= 0.4)}")


def checkGenIdxManual(events):
    delta_rs = events.Jet.delta_r(events.GenJet._apply_global_index(events.Jet.genJetIdxG)) 
    delta_rs =  delta_rs.compute()
    # print(delta_r_test)
    print(f"checkGenIdxManual: {ak.all(delta_rs <= 0.4)}")

def checkGenIdxSeparate(jets, genjets):
    matched_genjets = genjets._apply_global_index(jets.genJetIdxG)
    delta_rs = jets.delta_r(matched_genjets)
    # print(delta_rs)
    print(f"checkGenIdxOld: {ak.all(delta_rs <= 0.4)}")
    

In [8]:
jets.matched_gen

KeyError: '@events_factory'

In [7]:


checkGenIdxRecommended(events)
checkGenIdxManual(events)
jets = events.Jet.compute()
genjets = events.GenJet.compute()
checkGenIdxSeparate(jets, genjets)

checkGenIdxRecommended: True
checkGenIdxManual: True
checkGenIdxOld: False


In [11]:
df = events.compute()
gjj = ak.cartesian({"jet": df.Jet, "gjet": df.GenJet}, axis=1, nested=True)
_,_,deltar_gjj = delta_r(gjj["jet"].eta, gjj["gjet"].eta, gjj["jet"].phi, gjj["gjet"].phi)
# matched_jets = gjj[(deltar_gjj==deltar_gjj.min())&(deltar_gjj<0.4)].i1
mask = deltar_gjj == ak.min(deltar_gjj, axis=2)
# gjj_filtered = gjj[mask]
matched_jets = gjj[mask&(deltar_gjj<0.4)]["gjet"]
print(f"prepare_jets matched_jets: {matched_jets}")


/depot/cms/kernels/root632/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


prepare_jets matched_jets: [[[], [], [], []], ..., [[{eta: -0.257, mass: 7.52, phi: 2.99, ...}], ...]]


In [13]:
matched_jets

<PtEtaPhiMCollectionArray [[[], [], [], []], ..., [[{...}], ...]] type='20 ...'>

In [22]:
# ak.num(matched_jets, axis=1)
ak.num(ak.flatten(matched_jets, axis=-1), axis=1)

<Array [0, 1, 3, 3, 2, 1, 2, 4, ..., 4, 0, 2, 3, 4, 1, 3, 4] type='20 * int64'>

In [23]:
ak.flatten(matched_jets, axis=-1)

<PtEtaPhiMCollectionArray [[], ..., [{eta: -0.257, ...}, ...]] type='20 * v...'>

In [21]:
ak.sum(df.Jet.genJetIdx >= 0, axis=1)

<Array [0, 1, 3, 3, 2, 1, 2, 4, ..., 4, 0, 3, 3, 4, 1, 3, 4] type='20 * int64'>

In [16]:
print(ak.num(df.Jet, axis=1))
print(ak.num(df.GenJet, axis=1))

[4, 4, 5, 6, 4, 2, 5, 6, 6, 2, 3, 4, 4, 3, 5, 10, 6, 1, 5, 4]
[3, 2, 3, 4, 3, 2, 2, 4, 6, 3, 2, 5, 4, 0, 3, 3, 4, 2, 4, 6]


prepare_jets matched_jets: [[[], [], [], []], ..., [[{eta: -0.257, mass: 7.52, phi: 2.99, ...}], ...]]


In [37]:
# mask & (deltar_gjj<0.4)

AttributeError: no field named 'counts'

In [39]:
# matched_jets_flat = matched_jets.flatten()[matched_jets.flatten().counts>0,0]
matched_jets_flat = ak.flatten(matched_jets, axis=1)
matched_jets_flat

<PtEtaPhiMCollectionArray [[], ..., [{eta: 3.14, mass: 3.5, ...}]] type='89...'>

In [19]:
# ak.num(deltar_gjj,axis=2)
ak.min(deltar_gjj, axis=2)

<Array [[1.59, 3.32, 6.57, 0.832], ..., [...]] type='20 * var * ?float32'>

In [26]:
mask = deltar_gjj == ak.min(deltar_gjj, axis=2)
gjj_filtered = gjj[mask]

In [30]:
_,_,deltar_gjj_masked = delta_r(df.Jet.eta, gjj_filtered["gjet"].eta, df.Jet.phi, gjj_filtered["gjet"].phi)
deltar_gjj_masked

<Array [[[1.59], [3.32], ..., [0.832]], ...] type='20 * var * option[var * ...'>

In [21]:
# ak.min(deltar_gjj, axis=1)
deltar_gjj[0]

<Array [[3.44, 1.59, 2.11], ..., [0.832, ..., 2.35]] type='4 * var * float32'>

In [ ]:
matched_jets_flat = matched_jets.flatten()[matched_jets.flatten().counts>0,0]
matched_jets_new = awkward.JaggedArray.fromcounts((matched_jets.flatten().counts>0).astype(int), matched_jets_flat)
df["Jet",'matched_genjet'] = awkward.JaggedArray.fromcounts(matched_jets.counts,matched_jets_new)

In [1]:
import ROOT as rt

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
file = rt.TFile("data2018Snapshot.root")

In [3]:
file.ls()

TFile**		data2018Snapshot.root	out/data2018Snapshot.root
 TFile*		data2018Snapshot.root	out/data2018Snapshot.root
  KEY: TTree	Events;1	Events


In [12]:
file["Events"].get("DNN18AtanM1250")

AttributeError: 'TTree' object has no attribute 'get'

In [10]:
import uproot
uproot_file = uproot.open("data2018Snapshot.root")

In [18]:
uproot_file["Events"]["DNN18AtanM1250"].array()

<Array [0.0472, 0.291, 0.0981, ..., 0.162, 1.24, 1.02] type='3950 * float32'>